In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cupy
import numpy as np
import math
import time
import numba
from numba import cuda
from numba import njit
from numba import prange
import cudf
cupy.cuda.set_allocator(None)

N_PATHS = 16192000
N_STEPS = 365
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05

Single Thread CPU

In [ ]:
@njit(fastmath=True)
def cpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)
    running_average = 0.0
    for i in range(N_PATHS):
        s_curr = S0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break

        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [ ]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output =  np.zeros(N_PATHS, dtype=np.float32)

cpu_barrier_option(output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
s = time.time()
cpu_barrier_option(output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
v = output.mean()
e = time.time()
print('time', e-s, 'v', v)

time 10.073435068130493 v 18.709488


In [ ]:
import cupy as cp

# Calculate batch size (adjust target_mb_per_batch if needed)
elements_per_mb = 262144
target_mb_per_batch = 150
BATCH_SIZE = (target_mb_per_batch * elements_per_mb) // N_STEPS
BATCH_SIZE = min(BATCH_SIZE, 100000)  # Cap at 100k paths per batch

print(f"Processing with batch size: {BATCH_SIZE} paths")
output = np.zeros(N_PATHS, dtype=np.float32)

# Start timing for the whole computation
total_start = time.time()

running_sum = 0.0
processed_paths = 0

for i in range(0, N_PATHS, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, N_PATHS)
    batch_size = end_idx - i

    # Generate random numbers for this batch
    randoms_gpu = cp.random.normal(0, 1, batch_size * N_STEPS, dtype=cp.float32)
    randoms_cpu = cp.asnumpy(randoms_gpu)

    # Process batch
    batch_output = np.zeros(batch_size, dtype=np.float32)

    s = time.time()
    cpu_barrier_option(batch_output, np.float32(T), np.float32(K),
                      np.float32(B), np.float32(S0),
                      np.float32(sigma), np.float32(mu),
                      np.float32(r), randoms_cpu, N_STEPS, batch_size)

    # Store results and update running statistics
    output[i:end_idx] = batch_output
    running_sum += np.sum(batch_output)
    processed_paths += batch_size

    # Clean up GPU memory
    del randoms_gpu
    cp.get_default_memory_pool().free_all_blocks()

    # Print progress every 10 batches
    if (i // BATCH_SIZE) % 10 == 0:
        current_mean = running_sum / processed_paths
        print(f"Processed {processed_paths:,}/{N_PATHS:,} paths ({processed_paths/N_PATHS*100:.1f}%)")
        print(f"Current running mean: {current_mean:.6f}")

# Calculate final mean
final_mean = running_sum / N_PATHS
total_time = time.time() - total_start

print('='*50)
print(f'Total time: {total_time:.2f} seconds')
print(f'Final value: {final_mean:.6f}')

Processing with batch size: 100000 paths
Processed 100,000/3,192,000 paths (3.1%)
Current running mean: 18.709576
Processed 1,100,000/3,192,000 paths (34.5%)
Current running mean: 18.730733
Processed 2,100,000/3,192,000 paths (65.8%)
Current running mean: 18.743807
Processed 3,100,000/3,192,000 paths (97.1%)
Current running mean: 18.744738
Total time: 11.47 seconds
Final value: 18.741956


Multiple Cores CPU

In [ ]:
@njit(fastmath=True, parallel=True)
def cpu_multiplecore_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)

    # Pre-calculate 1/(n+1) values
    inv_n = np.array([1.0/(n + 1.0) for n in range(N_STEPS)])

    for i in prange(N_PATHS):
        s_curr = S0
        s_values = np.zeros(N_STEPS)
        barrier_hit = False

        # First pass: calculate all stock prices
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[n + i * N_STEPS]
            s_values[n] = s_curr

        # Second pass: calculate running average
        running_average = 0.0
        for n in range(N_STEPS):
            running_average = running_average + inv_n[n] * (s_values[n] - running_average)
            if running_average <= B:
                barrier_hit = True
                break

        payoff = running_average - K if (running_average > K and not barrier_hit) else 0
        d_s[i] = tmp2 * payoff

In [ ]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output =  np.zeros(N_PATHS, dtype=np.float32)
cpu_multiplecore_barrier_option(output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
s = time.time()
cpu_multiplecore_barrier_option(output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
v = output.mean()
e = time.time()
print('time', e-s, 'v', v)

time 3.929661273956299 v 18.728195


In [ ]:
# Calculate batch size
import cupy as cp
elements_per_mb = 262144
target_mb_per_batch = 150
BATCH_SIZE = (target_mb_per_batch * elements_per_mb) // N_STEPS
BATCH_SIZE = min(BATCH_SIZE, 100000)

print(f"Processing with batch size: {BATCH_SIZE} paths")
output = np.zeros(N_PATHS, dtype=np.float32)

# Compile the function (first call will compile)
small_test_size = 1000
test_output = np.zeros(small_test_size, dtype=np.float32)
test_randoms = np.random.normal(0, 1, small_test_size * N_STEPS).astype(np.float32)
cpu_multiplecore_barrier_option(test_output, np.float32(1.0), np.float32(100),
                          np.float32(120), np.float32(100),
                          np.float32(0.2), np.float32(0.05),
                          np.float32(0.05), test_randoms, N_STEPS, small_test_size)

# Start timing
total_start = time.time()
running_sum = 0.0
processed_paths = 0

for i in range(0, N_PATHS, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, N_PATHS)
    batch_size = end_idx - i

    # Generate random numbers for this batch
    randoms_gpu = cp.random.normal(0, 1, batch_size * N_STEPS, dtype=cp.float32)
    randoms_cpu = cp.asnumpy(randoms_gpu)

    # Process batch
    batch_output = np.zeros(batch_size, dtype=np.float32)

    s = time.time()
    cpu_multiplecore_barrier_option(batch_output, np.float32(T), np.float32(K),
                              np.float32(B), np.float32(S0),
                              np.float32(sigma), np.float32(mu),
                              np.float32(r), randoms_cpu, N_STEPS, batch_size)

    # Update results and statistics
    output[i:end_idx] = batch_output
    running_sum += np.sum(batch_output)
    processed_paths += batch_size

    # Clean up GPU memory
    del randoms_gpu
    cp.get_default_memory_pool().free_all_blocks()

    # Print progress
    if (i // BATCH_SIZE) % 10 == 0:
        current_mean = running_sum / processed_paths
        elapsed = time.time() - total_start
        paths_per_second = processed_paths / elapsed
        print(f"Processed {processed_paths:,}/{N_PATHS:,} paths ({processed_paths/N_PATHS*100:.1f}%)")
        print(f"Current mean: {current_mean:.6f}")
        print(f"Processing speed: {paths_per_second:.0f} paths/second")

final_mean = running_sum / N_PATHS
total_time = time.time() - total_start

print('='*50)
print(f'Total time: {total_time:.2f} seconds')
print(f'Final value: {final_mean:.6f}')
print(f'Average processing speed: {N_PATHS/total_time:.0f} paths/second')

Processing with batch size: 100000 paths
Processed 100,000/3,192,000 paths (3.1%)
Current mean: 18.708804
Processing speed: 607406 paths/second
Processed 1,100,000/3,192,000 paths (34.5%)
Current mean: 18.693948
Processing speed: 575533 paths/second
Processed 2,100,000/3,192,000 paths (65.8%)
Current mean: 18.718199
Processing speed: 551608 paths/second
Processed 3,100,000/3,192,000 paths (97.1%)
Current mean: 18.716147
Processing speed: 575150 paths/second
Total time: 5.53 seconds
Final value: 18.716864
Average processing speed: 576910 paths/second


NUMBA GPU

In [ ]:
@cuda.jit
def numba_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):  # Make sure all parameters are here
    # Get thread ID
    tid = cuda.grid(1)  # 1D grid

    # Check if thread is within bounds
    if tid < N_PATHS:
        s_curr = S0
        running_average = 0.0
        for n in range(N_STEPS):
            s_curr += mu*T/N_STEPS * s_curr + sigma*s_curr*math.sqrt(T/N_STEPS)*d_normals[tid + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break
        payoff = running_average - K if running_average > K else 0
        d_s[tid] = math.exp(-r*T) * payoff

In [ ]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output =  np.zeros(N_PATHS, dtype=np.float32)
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
numba_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
s = time.time()
numba_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v)

time 0.48715853691101074 v 18.705944


In [ ]:
# Calculate batch size
import cupy as cp
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
elements_per_mb = 262144
target_mb_per_batch = 150
BATCH_SIZE = (target_mb_per_batch * elements_per_mb) // N_STEPS
BATCH_SIZE = min(BATCH_SIZE, 100000)

print(f"Processing with batch size: {BATCH_SIZE} paths")
output = np.zeros(N_PATHS, dtype=np.float32)

# Compile the function (first call will compile)
small_test_size = 1000
test_output = np.zeros(small_test_size, dtype=np.float32)
test_randoms = np.random.normal(0, 1, small_test_size * N_STEPS).astype(np.float32)
numba_barrier_option[(number_of_blocks,), (number_of_threads,)](test_output, np.float32(1.0), np.float32(100),
                          np.float32(120), np.float32(100),
                          np.float32(0.2), np.float32(0.05),
                          np.float32(0.05), test_randoms, N_STEPS, small_test_size)

# Start timing
total_start = time.time()
running_sum = 0.0
processed_paths = 0

for i in range(0, N_PATHS, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, N_PATHS)
    batch_size = end_idx - i

    # Generate random numbers for this batch
    randoms_gpu = cp.random.normal(0, 1, batch_size * N_STEPS, dtype=cp.float32)
    randoms_cpu = cp.asnumpy(randoms_gpu)

    # Process batch
    batch_output = np.zeros(batch_size, dtype=np.float32)

    s = time.time()
    numba_barrier_option[(number_of_blocks,), (number_of_threads,)](batch_output, np.float32(T), np.float32(K),
                              np.float32(B), np.float32(S0),
                              np.float32(sigma), np.float32(mu),
                              np.float32(r), randoms_cpu, N_STEPS, batch_size)

    # Update results and statistics
    output[i:end_idx] = batch_output
    running_sum += np.sum(batch_output)
    processed_paths += batch_size

    # Clean up GPU memory
    del randoms_gpu
    cp.get_default_memory_pool().free_all_blocks()

    # Print progress
    if (i // BATCH_SIZE) % 10 == 0:
        current_mean = running_sum / processed_paths
        elapsed = time.time() - total_start
        paths_per_second = processed_paths / elapsed
        print(f"Processed {processed_paths:,}/{N_PATHS:,} paths ({processed_paths/N_PATHS*100:.1f}%)")
        print(f"Current mean: {current_mean:.6f}")
        print(f"Processing speed: {paths_per_second:.0f} paths/second")

final_mean = running_sum / N_PATHS
total_time = time.time() - total_start

print('='*50)
print(f'Total time: {total_time:.2f} seconds')
print(f'Final value: {final_mean:.6f}')
print(f'Average processing speed: {N_PATHS/total_time:.0f} paths/second')

Processing with batch size: 100000 paths


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Processed 100,000/3,192,000 paths (3.1%)
Current mean: 18.592958
Processing speed: 687307 paths/second
Processed 1,100,000/3,192,000 paths (34.5%)
Current mean: 18.699063
Processing speed: 705193 paths/second
Processed 2,100,000/3,192,000 paths (65.8%)
Current mean: 18.703537
Processing speed: 716049 paths/second
Processed 3,100,000/3,192,000 paths (97.1%)
Current mean: 18.702760
Processing speed: 722328 paths/second
Total time: 4.41 seconds
Final value: 18.700663
Average processing speed: 723021 paths/second


CUPY GPU

In [ ]:
cupy_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void barrier_option(
    float *d_s,
    const float T,
    const float K,
    const float B,
    const float S0,
    const float sigma,
    const float mu,
    const float r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS)
{
    unsigned idx = threadIdx.x + blockIdx.x * blockDim.x;
    unsigned stride = blockDim.x * gridDim.x;

    // Precompute constants
    const float tmp1 = mu * T / N_STEPS;
    const float tmp2 = exp(-r * T);
    const float tmp3 = sqrt(T / N_STEPS);

    // Use shared memory for frequently accessed data
    __shared__ float s_temp[32];  // Adjust size based on your block size

    for (unsigned i = idx; i < N_PATHS; i += stride) {
        float s_curr = S0;
        float running_average = 0.0f;  // Changed to float for consistency

        // Reorganized memory access pattern
        const unsigned base_idx = i * N_STEPS;

        for(unsigned n = 0; n < N_STEPS; n++) {
            s_curr += tmp1 * s_curr + sigma * s_curr * tmp3 * d_normals[base_idx + n];
            running_average += (s_curr - running_average) / (n + 1.0f);

            if (running_average <= B) {
                break;
            }
        }

        float payoff = (running_average > K) ? (running_average - K) : 0.0f;
        d_s[i] = tmp2 * payoff;
    }
}
''', 'barrier_option')

In [ ]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
s = time.time()
cupy_barrier_option((number_of_blocks,), (number_of_threads,),
                   (output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r),  randoms_gpu, N_STEPS, N_PATHS))
v = output.mean()
cupy.cuda.stream.get_current_stream().synchronize()
e = time.time()
print('time', e-s, 'v',v)

In [ ]:
import cupy as cp
import time
import numpy as np

# Calculate batch size
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cp.zeros(N_PATHS, dtype=cp.float32)
elements_per_mb = 262144
target_mb_per_batch = 150
BATCH_SIZE = (target_mb_per_batch * elements_per_mb) // N_STEPS
BATCH_SIZE = min(BATCH_SIZE, 100000)

print(f"Processing with batch size: {BATCH_SIZE} paths")

# Compile the function (first call will compile)
small_test_size = 1000
test_output = cp.zeros(small_test_size, dtype=cp.float32)
test_randoms = cp.random.normal(0, 1, small_test_size * N_STEPS, dtype=cp.float32)

# Test kernel launch
number_of_blocks_test = (small_test_size + number_of_threads - 1) // number_of_threads
cupy_barrier_option((number_of_blocks_test,), (number_of_threads,),(
    test_output,
    np.float32(1.0),
    np.float32(100),
    np.float32(120),
    np.float32(100),
    np.float32(0.2),
    np.float32(0.05),
    np.float32(0.05),
    test_randoms,
    N_STEPS,
    small_test_size
))

# Start timing
total_start = time.time()
running_sum = 0.0
processed_paths = 0

for i in range(0, N_PATHS, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, N_PATHS)
    batch_size = end_idx - i

    # Calculate blocks for this batch
    blocks_for_batch = (batch_size + number_of_threads - 1) // number_of_threads

    # Generate random numbers directly on GPU
    randoms_gpu = cp.random.normal(0, 1, batch_size * N_STEPS, dtype=cp.float32)

    # Allocate output on GPU
    batch_output_gpu = cp.zeros(batch_size, dtype=cp.float32)

    s = time.time()
    cupy_barrier_option((blocks_for_batch,), (number_of_threads,),(
        batch_output_gpu,
        np.float32(T),
        np.float32(K),
        np.float32(B),
        np.float32(S0),
        np.float32(sigma),
        np.float32(mu),
        np.float32(r),
        randoms_gpu,
        N_STEPS,
        batch_size
    ))

    # Copy results back to CPU only if needed for statistics
    batch_output = cp.asnumpy(batch_output_gpu)

    # Update results and statistics
    output[i:end_idx] = batch_output_gpu
    running_sum += float(cp.sum(batch_output_gpu))  # Convert CuPy sum to Python float
    processed_paths += batch_size

    # Clean up GPU memory
    del randoms_gpu
    del batch_output_gpu
    cp.get_default_memory_pool().free_all_blocks()

    # Print progress
    if (i // BATCH_SIZE) % 10 == 0:
        current_mean = running_sum / processed_paths
        elapsed = time.time() - total_start
        paths_per_second = processed_paths / elapsed
        # print(f"Processed {processed_paths:,}/{N_PATHS:,} paths ({processed_paths/N_PATHS*100:.1f}%)")
        # print(f"Current mean: {current_mean:.6f}")
        # print(f"Processing speed: {paths_per_second:.0f} paths/second")

final_mean = running_sum / N_PATHS
total_time = time.time() - total_start

print('='*50)
print(f'Total time: {total_time:.2f} seconds')
print(f'Final value: {final_mean:.6f}')
print(f'Average processing speed: {N_PATHS/total_time:.0f} paths/second')

Processing with batch size: 100000 paths
Total time: 1.82 seconds
Final value: 18.709442
Average processing speed: 8920861 paths/second
